In [1]:
from src import angles as ang
from fractions import Fraction as Frac

In [84]:
def coreEntropy(num, den):
    #define a new rational angle
    theta = ang.Angle(num,den)
    
    thetaFr = Frac(theta.num,theta.den)#represent it as a fraction
   
    #compute the orbit and find the period
    theta.period()
    orb = theta.orbit_list
    period_length = theta.per_len
    period_start = theta.start_index_per
    
    #partition of the circle
    intOne = (thetaFr*Frac(1,2),(thetaFr+Frac(1,1))*Frac(1,2))
    
    print(f"The angle is {thetaFr} \n which has a period of {period_length} \n and the porbit is {orb}\n")
    print(f"partition the circle in two intervals: {intOne} and {(intOne[1],intOne[0])}\n")
    
    #create the vertex set of the wedge
    tuples = sorted([(i,j) for i in range(1,len(orb[:-1])) for j in range(1,len(orb[:-1])) if i<=j])
    #define the separated and non-separated vertices
    dicTuples = {}
    for tup in tuples:
        i=tup[0]
        j=tup[1]
        print(f"the tuple {tup} corresponding to the angles {orb[i-1]} and {orb[j-1]}")
        if i==j:
            print("is not separated\n")
            dicTuples.update({tup:{'sep':False,'mapsTo':[(i+1,j+1)]}})
        elif ((intOne[0]<orb[i-1]<intOne[1] and intOne[0]<orb[j-1]<intOne[1]) or (intOne[1]<=orb[i-1] and intOne[1]<=orb[j-1])):
            print("is not separated\n")
            dicTuples.update({tup:{'sep':False,'mapsTo':[(i+1,j+1)]}})
        else:
            print("is separated\n")
            dicTuples.update({tup:{'sep':True,'mapsTo':[(1,i+1),(1,j+1)]}})
    print(dicTuples)
            
    

In [87]:
coreEntropy(3,7)

The angle is 3/7 
 which has a period of 3 
 and the porbit is [Fraction(3, 7), Fraction(6, 7), Fraction(5, 7), Fraction(3, 7)]

partition the circle in two intervals: (Fraction(3, 14), Fraction(5, 7)) and (Fraction(5, 7), Fraction(3, 14))

the tuple (1, 1) corresponding to the angles 3/7 and 3/7
is not separated

the tuple (1, 2) corresponding to the angles 3/7 and 6/7
is separated

the tuple (2, 2) corresponding to the angles 6/7 and 6/7
is not separated

{(1, 1): {'sep': False, 'mapsTo': [(2, 2)]}, (1, 2): {'sep': True, 'mapsTo': [(1, 2), (1, 3)]}, (2, 2): {'sep': False, 'mapsTo': [(3, 3)]}}
